In [29]:
#import Libriaries
import xml.etree.ElementTree as ET
import pandas as pd
import re
import pyodbc #oldwayto connect to sql server
import pydash #to format table and field names
import pandasgui as pg #to visualize dataframes
import sqlalchemy  #new way to connect to sql server
import os 

In [11]:
# Define the connection details
server = 'jacobo-dev.database.windows.net'
port = '1433'
database = 'jacobo-dev-sqlserver-azure-001'
username = 'azure-admin'
password = 'ja-2023-un0ypzjo'
driver = '{ODBC Driver 18 for SQL Server}'

# Define the connection string
conn_str = f"DRIVER={driver};SERVER={server},{port};DATABASE={database};UID={username};PWD={password};Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30"

In [12]:
# Test SQL DB Connection
try:
    conn = pyodbc.connect(conn_str)
    print("Connection successful!")
    conn.close()
except pyodbc.Error as e:
    print("Error connecting to database:", e)


Connection successful!


In [13]:
#function to get all the child elements of a given xml element

def get_child_elements(xml_root):
    # Create a list to store all child elements
    elements = []
    #append xml root to elements list
    #elements.append(xml_root)
    # Get all child elements of a given xml element
    for child in xml_root:
        elements.append(child)
        elements.extend(get_child_elements(child))
    return elements


In [14]:
def process_xml_elements(conn, elements):
    """
    Inserts the attributes of each XML element into a SQL table with the same name
    as the element tag.
    """
    #conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()
    
    # loop through the elements and their attributes
    for element in elements:
        # create the SQL table name using the element tag
        table_name = element.tag

        # create the SQL create table query
        create_table_query = f"CREATE TABLE IF NOT EXISTS {table_name} ("
        for attr in element.attrib:
            create_table_query += f"{attr} VARCHAR(MAX), "
        create_table_query = create_table_query[:-2] + ")"

        # execute the create table query
        cursor.execute(create_table_query)
        cursor.commit()

        # insert the element attributes into the table
        insert_query = f"INSERT INTO {table_name} ("
        for attr in element.attrib:
            insert_query += f"{attr}, "
        insert_query = insert_query[:-2] + ") VALUES ("
        for attr in element.attrib:
            insert_query += f"'{element.attrib[attr]}', "
        insert_query = insert_query[:-2] + ")"

        # execute the insert query
        cursor.execute(insert_query)
        cursor.commit()

    # close the database connection
    conn.close()

In [15]:
def pascal_case(text):
    return ''.join(word.capitalize() for word in text.split())


In [16]:
# Define file to read
xml_file_path = r"C:\Users\Roberto\OneDrive\cargoIabono\Proyectos y Desarrollos\P001_V001_CFDI-Reader\01_Inputs\Nomina\EMS2103108P3_Pago de nómina_20220815_N_AOAR951019842.xml"
xml_tree = ET.parse(xml_file_path)
xml_root = xml_tree.getroot()

In [17]:
#Read XML Child elements
elements = get_child_elements(xml_root)

#add xml root to read it with their child elements
elements.append(xml_root)

#print child elements
for element in elements:
    if element.attrib:
        print(element.tag, element.attrib)

{http://www.sat.gob.mx/cfd/3}Emisor {'Rfc': 'EMS2103108P3', 'Nombre': 'ESAB MEXICO SERVICES SA DE CV', 'RegimenFiscal': '601'}
{http://www.sat.gob.mx/cfd/3}Receptor {'Rfc': 'AOAR951019842', 'Nombre': 'Roberto Carlos Alonso Alvarez', 'UsoCFDI': 'P01'}
{http://www.sat.gob.mx/cfd/3}Concepto {'ClaveProdServ': '84111505', 'Cantidad': '1', 'ClaveUnidad': 'ACT', 'Descripcion': 'Pago de nómina', 'ValorUnitario': '43376.49', 'Importe': '43376.49', 'Descuento': '14787.52'}
{http://www.sat.gob.mx/nomina12}Nomina {'Version': '1.2', 'TipoNomina': 'O', 'FechaPago': '2022-08-15', 'FechaInicialPago': '2022-08-01', 'FechaFinalPago': '2022-08-15', 'NumDiasPagados': '15.000', 'TotalPercepciones': '43376.49', 'TotalDeducciones': '14787.52', 'TotalOtrosPagos': '0.00'}
{http://www.sat.gob.mx/nomina12}Emisor {'RegistroPatronal': 'Y3936393101'}
{http://www.sat.gob.mx/nomina12}Receptor {'Curp': 'AOAR951019HNLLLB06', 'NumSeguridadSocial': '43129510616', 'FechaInicioRelLaboral': '2022-02-01', 'Antigüedad': 'P28W

**SQL Query with pandas.**

In [18]:
## Test Query Retrieve Data

# Define the SQL query
sample_query = "SELECT * FROM [cfditest].[SampleTable]"

# Establish the database connection
conn = pyodbc.connect(conn_str)

# Execute the SQL query and load the data into a pandas dataframe
df = pd.read_sql(sample_query, conn)

# Print the first five rows of the dataframe
print(df.head())

# display data using pandasgui
pg.show(df.head())

C:\Users\Roberto\AppData\Local\Temp\ipykernel_17688\311658007.py:10: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

PandasGUI INFO — pandasgui.gui — Opening PandasGUI


   Id         Name  Age                    Email
0   1     John Doe   30     john.doe@example.com
1   2   Jane Smith   25   jane.smith@example.com
2   3  Bob Johnson   40  bob.johnson@example.com
3   4    Alice Lee   35    alice.lee@example.com
4   5    Tom Brown   28    tom.brown@example.com


In [19]:
#cleantag
for element in elements:
    cleaned_tag = re.sub(r"{.*?}", "", element.tag)
    print(cleaned_tag)

Emisor
Receptor
Conceptos
Concepto
Complemento
Nomina
Emisor
Receptor
Percepciones
Percepcion
Percepcion
Deducciones
Deduccion
Deduccion
Deduccion
Deduccion
OtrosPagos
OtroPago
SubsidioAlEmpleo
TimbreFiscalDigital
Comprobante


**Bard**

In [23]:

# Create a DataFrame
df = pd.DataFrame()
for child in xml_root:

    # Get the tag and text of the child element
    tag = child.tag
    text = child.text

    # Add the data to the DataFrame
    df = df.append({'tag': tag, 'text': text}, ignore_index=True)

# Show the DataFrame
df

C:\Users\Roberto\AppData\Local\Temp\ipykernel_17688\2280181311.py:10: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Roberto\AppData\Local\Temp\ipykernel_17688\2280181311.py:10: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Roberto\AppData\Local\Temp\ipykernel_17688\2280181311.py:10: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Roberto\AppData\Local\Temp\ipykernel_17688\2280181311.py:10: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



,tag,text
0,{http://www.sat.gob.mx/cfd/3}Emisor,None
1,{http://www.sat.gob.mx/cfd/3}Receptor,None
2,{http://www.sat.gob.mx/cfd/3}Conceptos,\n\t\t
3,{http://www.sat.gob.mx/cfd/3}Complemento,\n\t\t


In [26]:
#print child elements
for element in elements:
    if element.attrib:
        cleaned_tag = re.sub(r"{.*?}", "", element.tag)
        print(cleaned_tag)
        print(element.tag, element.attrib)

Emisor
{http://www.sat.gob.mx/cfd/3}Emisor {'Rfc': 'EMS2103108P3', 'Nombre': 'ESAB MEXICO SERVICES SA DE CV', 'RegimenFiscal': '601'}
Receptor
{http://www.sat.gob.mx/cfd/3}Receptor {'Rfc': 'AOAR951019842', 'Nombre': 'Roberto Carlos Alonso Alvarez', 'UsoCFDI': 'P01'}
Concepto
{http://www.sat.gob.mx/cfd/3}Concepto {'ClaveProdServ': '84111505', 'Cantidad': '1', 'ClaveUnidad': 'ACT', 'Descripcion': 'Pago de nómina', 'ValorUnitario': '43376.49', 'Importe': '43376.49', 'Descuento': '14787.52'}
Nomina
{http://www.sat.gob.mx/nomina12}Nomina {'Version': '1.2', 'TipoNomina': 'O', 'FechaPago': '2022-08-15', 'FechaInicialPago': '2022-08-01', 'FechaFinalPago': '2022-08-15', 'NumDiasPagados': '15.000', 'TotalPercepciones': '43376.49', 'TotalDeducciones': '14787.52', 'TotalOtrosPagos': '0.00'}
Emisor
{http://www.sat.gob.mx/nomina12}Emisor {'RegistroPatronal': 'Y3936393101'}
Receptor
{http://www.sat.gob.mx/nomina12}Receptor {'Curp': 'AOAR951019HNLLLB06', 'NumSeguridadSocial': '43129510616', 'FechaInic

In [34]:
#print child elements
for element in elements:
    if element.attrib:
        cleaned_tag = re.sub(r"{.*?}", "", element.tag)
        file_name = os.path.basename(xml_file_path)
        print(file_name)
        print(cleaned_tag, element.tag, element.attrib)

EMS2103108P3_Pago de nómina_20220815_N_AOAR951019842.xml
Emisor {http://www.sat.gob.mx/cfd/3}Emisor {'Rfc': 'EMS2103108P3', 'Nombre': 'ESAB MEXICO SERVICES SA DE CV', 'RegimenFiscal': '601'}
EMS2103108P3_Pago de nómina_20220815_N_AOAR951019842.xml
Receptor {http://www.sat.gob.mx/cfd/3}Receptor {'Rfc': 'AOAR951019842', 'Nombre': 'Roberto Carlos Alonso Alvarez', 'UsoCFDI': 'P01'}
EMS2103108P3_Pago de nómina_20220815_N_AOAR951019842.xml
Concepto {http://www.sat.gob.mx/cfd/3}Concepto {'ClaveProdServ': '84111505', 'Cantidad': '1', 'ClaveUnidad': 'ACT', 'Descripcion': 'Pago de nómina', 'ValorUnitario': '43376.49', 'Importe': '43376.49', 'Descuento': '14787.52'}
EMS2103108P3_Pago de nómina_20220815_N_AOAR951019842.xml
Nomina {http://www.sat.gob.mx/nomina12}Nomina {'Version': '1.2', 'TipoNomina': 'O', 'FechaPago': '2022-08-15', 'FechaInicialPago': '2022-08-01', 'FechaFinalPago': '2022-08-15', 'NumDiasPagados': '15.000', 'TotalPercepciones': '43376.49', 'TotalDeducciones': '14787.52', 'TotalOtr

In [43]:
#print child elements
for element in elements:
    if element.attrib:
        keys = elements[0].attrib.keys()
        values = elements[0].attrib.values()
        cleaned_tag = re.sub(r"{.*?}", "", element.tag)
        print("Cleaned Tag:", cleaned_tag)
        #print("Tag:", element.tag)
        print("Keys:", keys)
        print("Values:", values)
        #print("Element:", element.attrib)

Cleaned Tag: Emisor
Keys: dict_keys(['Rfc', 'Nombre', 'RegimenFiscal'])
Values: dict_values(['EMS2103108P3', 'ESAB MEXICO SERVICES SA DE CV', '601'])
Cleaned Tag: Receptor
Keys: dict_keys(['Rfc', 'Nombre', 'RegimenFiscal'])
Values: dict_values(['EMS2103108P3', 'ESAB MEXICO SERVICES SA DE CV', '601'])
Cleaned Tag: Concepto
Keys: dict_keys(['Rfc', 'Nombre', 'RegimenFiscal'])
Values: dict_values(['EMS2103108P3', 'ESAB MEXICO SERVICES SA DE CV', '601'])
Cleaned Tag: Nomina
Keys: dict_keys(['Rfc', 'Nombre', 'RegimenFiscal'])
Values: dict_values(['EMS2103108P3', 'ESAB MEXICO SERVICES SA DE CV', '601'])
Cleaned Tag: Emisor
Keys: dict_keys(['Rfc', 'Nombre', 'RegimenFiscal'])
Values: dict_values(['EMS2103108P3', 'ESAB MEXICO SERVICES SA DE CV', '601'])
Cleaned Tag: Receptor
Keys: dict_keys(['Rfc', 'Nombre', 'RegimenFiscal'])
Values: dict_values(['EMS2103108P3', 'ESAB MEXICO SERVICES SA DE CV', '601'])
Cleaned Tag: Percepciones
Keys: dict_keys(['Rfc', 'Nombre', 'RegimenFiscal'])
Values: dict_val

In [47]:
#print child elements
for element in elements:
    if element.attrib:
        cleaned_tag = re.sub(r"{.*?}", "", element.tag)
        file_name = os.path.basename(xml_file_path)
        print(file_name)
        print(cleaned_tag, element.tag, element.attrib)
        print("Cleaned Tag:", cleaned_tag)
        print("Tag:", element.tag)
        keys = element.attrib.keys()
        values = element.attrib.values()
        print("Keys:", keys)
        print("Values:", values)


Cleaned Tag: Emisor
Tag: {http://www.sat.gob.mx/cfd/3}Emisor
Keys: dict_keys(['Rfc', 'Nombre', 'RegimenFiscal'])
Values: dict_values(['EMS2103108P3', 'ESAB MEXICO SERVICES SA DE CV', '601'])
Cleaned Tag: Receptor
Tag: {http://www.sat.gob.mx/cfd/3}Receptor
Keys: dict_keys(['Rfc', 'Nombre', 'UsoCFDI'])
Values: dict_values(['AOAR951019842', 'Roberto Carlos Alonso Alvarez', 'P01'])
Cleaned Tag: Concepto
Tag: {http://www.sat.gob.mx/cfd/3}Concepto
Keys: dict_keys(['ClaveProdServ', 'Cantidad', 'ClaveUnidad', 'Descripcion', 'ValorUnitario', 'Importe', 'Descuento'])
Values: dict_values(['84111505', '1', 'ACT', 'Pago de nómina', '43376.49', '43376.49', '14787.52'])
Cleaned Tag: Nomina
Tag: {http://www.sat.gob.mx/nomina12}Nomina
Keys: dict_keys(['Version', 'TipoNomina', 'FechaPago', 'FechaInicialPago', 'FechaFinalPago', 'NumDiasPagados', 'TotalPercepciones', 'TotalDeducciones', 'TotalOtrosPagos'])
Values: dict_values(['1.2', 'O', '2022-08-15', '2022-08-01', '2022-08-15', '15.000', '43376.49', '14

In [48]:
# Create a DataFrame
df = pd.DataFrame()

#print child elements
for element in elements:
    if element.attrib:
        cleaned_tag = re.sub(r"{.*?}", "", element.tag)
        file_name = os.path.basename(xml_file_path)
        keys = element.attrib.keys()
        values = element.attrib.values()
        
        # Add the data to the DataFrame
        df = df.append({'file_name': file_name,'cleaned_tag': cleaned_tag,'keys': keys, 'values': values}, ignore_index=True)

# Show the DataFrame
df
        
        

C:\Users\Roberto\AppData\Local\Temp\ipykernel_17688\3813379824.py:13: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Roberto\AppData\Local\Temp\ipykernel_17688\3813379824.py:13: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Roberto\AppData\Local\Temp\ipykernel_17688\3813379824.py:13: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Roberto\AppData\Local\Temp\ipykernel_17688\3813379824.py:13: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Roberto\AppData\Local\Temp\ipykernel_17688\3813379824.py:13: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use

,file_name,cleaned_tag,keys,values
0,EMS2103108P3_Pago de nómina_20220815_N_AOAR951...,Emisor,"(Rfc, Nombre, RegimenFiscal)","(EMS2103108P3, ESAB MEXICO SERVICES SA DE CV, ..."
1,EMS2103108P3_Pago de nómina_20220815_N_AOAR951...,Receptor,"(Rfc, Nombre, UsoCFDI)","(AOAR951019842, Roberto Carlos Alonso Alvarez,..."
2,EMS2103108P3_Pago de nómina_20220815_N_AOAR951...,Concepto,"(ClaveProdServ, Cantidad, ClaveUnidad, Descrip...","(84111505, 1, ACT, Pago de nómina, 43376.49, 4..."
3,EMS2103108P3_Pago de nómina_20220815_N_AOAR951...,Nomina,"(Version, TipoNomina, FechaPago, FechaInicialP...","(1.2, O, 2022-08-15, 2022-08-01, 2022-08-15, 1..."
4,EMS2103108P3_Pago de nómina_20220815_N_AOAR951...,Emisor,(RegistroPatronal),(Y3936393101)
5,EMS2103108P3_Pago de nómina_20220815_N_AOAR951...,Receptor,"(Curp, NumSeguridadSocial, FechaInicioRelLabor...","(AOAR951019HNLLLB06, 43129510616, 2022-02-01, ..."
6,EMS2103108P3_Pago de nómina_20220815_N_AOAR951...,Percepciones,"(TotalSueldos, TotalGravado, TotalExento)","(43376.49, 41500.20, 1876.29)"
7,EMS2103108P3_Pago de nómina_20220815_N_AOAR951...,Percepcion,"(TipoPercepcion, Clave, Concepto, ImporteGrava...","(001, 101, Sueldo normal, 41500.2, 0)"
8,EMS2103108P3_Pago de nómina_20220815_N_AOAR951...,Percepcion,"(TipoPercepcion, Clave, Concepto, ImporteGrava...","(005, 1fa, Fondo de ahorro, 0, 1876.29)"
9,EMS2103108P3_Pago de nómina_20220815_N_AOAR951...,Deducciones,"(TotalOtrasDeducciones, TotalImpuestosRetenidos)","(4709.84, 10077.68)"


In [58]:
# Create a DataFrame
df = pd.DataFrame()

#print child elements
for element in elements:
    if element.attrib:
        cleaned_tag = re.sub(r"{.*?}", "", element.tag)
        file_name = os.path.basename(xml_file_path)
        keys = element.attrib.keys()
        values = element.attrib.values()
        
        # Add the data to the DataFrame
        df = df.append({'file_name': file_name,'cleaned_tag': cleaned_tag,'keys': keys, 'values': values}, ignore_index=True)

# Show the DataFrame
df
        

C:\Users\Roberto\AppData\Local\Temp\ipykernel_17688\4234174519.py:13: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Roberto\AppData\Local\Temp\ipykernel_17688\4234174519.py:13: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Roberto\AppData\Local\Temp\ipykernel_17688\4234174519.py:13: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Roberto\AppData\Local\Temp\ipykernel_17688\4234174519.py:13: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Roberto\AppData\Local\Temp\ipykernel_17688\4234174519.py:13: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use

,file_name,cleaned_tag,keys,values
0,EMS2103108P3_Pago de nómina_20220815_N_AOAR951...,Emisor,"(Rfc, Nombre, RegimenFiscal)","(EMS2103108P3, ESAB MEXICO SERVICES SA DE CV, ..."
1,EMS2103108P3_Pago de nómina_20220815_N_AOAR951...,Receptor,"(Rfc, Nombre, UsoCFDI)","(AOAR951019842, Roberto Carlos Alonso Alvarez,..."
2,EMS2103108P3_Pago de nómina_20220815_N_AOAR951...,Concepto,"(ClaveProdServ, Cantidad, ClaveUnidad, Descrip...","(84111505, 1, ACT, Pago de nómina, 43376.49, 4..."
3,EMS2103108P3_Pago de nómina_20220815_N_AOAR951...,Nomina,"(Version, TipoNomina, FechaPago, FechaInicialP...","(1.2, O, 2022-08-15, 2022-08-01, 2022-08-15, 1..."
4,EMS2103108P3_Pago de nómina_20220815_N_AOAR951...,Emisor,(RegistroPatronal),(Y3936393101)
5,EMS2103108P3_Pago de nómina_20220815_N_AOAR951...,Receptor,"(Curp, NumSeguridadSocial, FechaInicioRelLabor...","(AOAR951019HNLLLB06, 43129510616, 2022-02-01, ..."
6,EMS2103108P3_Pago de nómina_20220815_N_AOAR951...,Percepciones,"(TotalSueldos, TotalGravado, TotalExento)","(43376.49, 41500.20, 1876.29)"
7,EMS2103108P3_Pago de nómina_20220815_N_AOAR951...,Percepcion,"(TipoPercepcion, Clave, Concepto, ImporteGrava...","(001, 101, Sueldo normal, 41500.2, 0)"
8,EMS2103108P3_Pago de nómina_20220815_N_AOAR951...,Percepcion,"(TipoPercepcion, Clave, Concepto, ImporteGrava...","(005, 1fa, Fondo de ahorro, 0, 1876.29)"
9,EMS2103108P3_Pago de nómina_20220815_N_AOAR951...,Deducciones,"(TotalOtrasDeducciones, TotalImpuestosRetenidos)","(4709.84, 10077.68)"


In [60]:

# Iterate through the tree and extract the attributes of each element.
attributes = []
for element in elements:
    attributes.append(element.attrib)

# Create a pandas DataFrame with the extracted attributes.
df1 = pd.DataFrame(attributes)

# Print the DataFrame.
df1

,Rfc,Nombre,RegimenFiscal,UsoCFDI,ClaveProdServ,Cantidad,ClaveUnidad,Descripcion,ValorUnitario,Importe,...,Sello,FormaPago,NoCertificado,Certificado,SubTotal,Moneda,Total,TipoDeComprobante,MetodoPago,LugarExpedicion
0,EMS2103108P3,ESAB MEXICO SERVICES SA DE CV,601,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AOAR951019842,Roberto Carlos Alonso Alvarez,NaN,P01,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,84111505,1,ACT,Pago de nómina,43376.49,43376.49,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:


# Define a list to store row data
data = []

# Iterate over elements
for element in elements:
    if element.attrib:
        cleaned_tag = re.sub(r"{.*?}", "", element.tag)
        file_name = os.path.basename(xml_file_path)
        
        # Iterate over attributes of the element
        for key, value in element.attrib.items():
            # Add the data to the list
            data.append({'file_name': file_name, 'cleaned_tag': cleaned_tag, 'key': key, 'value': value})

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(data)

# Show the DataFrame
df

,file_name,cleaned_tag,key,value
0,EMS2103108P3_Pago de nómina_20220815_N_AOAR951...,Emisor,Rfc,EMS2103108P3
1,EMS2103108P3_Pago de nómina_20220815_N_AOAR951...,Emisor,Nombre,ESAB MEXICO SERVICES SA DE CV
2,EMS2103108P3_Pago de nómina_20220815_N_AOAR951...,Emisor,RegimenFiscal,601
3,EMS2103108P3_Pago de nómina_20220815_N_AOAR951...,Receptor,Rfc,AOAR951019842
4,EMS2103108P3_Pago de nómina_20220815_N_AOAR951...,Receptor,Nombre,Roberto Carlos Alonso Alvarez
...,...,...,...,...
96,EMS2103108P3_Pago de nómina_20220815_N_AOAR951...,Comprobante,Moneda,MXN
97,EMS2103108P3_Pago de nómina_20220815_N_AOAR951...,Comprobante,Total,28588.97
98,EMS2103108P3_Pago de nómina_20220815_N_AOAR951...,Comprobante,TipoDeComprobante,N
99,EMS2103108P3_Pago de nómina_20220815_N_AOAR951...,Comprobante,MetodoPago,PUE


In [75]:

# Define a list to store row data
data = []

# Define a dictionary for the attribute field counters
field_counters = {}

# Variable to store the UUID
uuid = None

# Iterate over elements
for element in elements:
    if element.attrib:
        cleaned_tag = re.sub(r"{.*?}", "", element.tag)
        file_name = os.path.basename('xml_file_path')

        # If this tag has not been seen before, initialize its counter
        if element.tag not in field_counters:
            field_counters[element.tag] = 1

        # Check if the element is 'TimbreFiscalDigital' and then extract the UUID
        if cleaned_tag == 'TimbreFiscalDigital':
            uuid = element.attrib.get('UUID')

        # Iterate over attributes of the element
        for key, value in element.attrib.items():
            # Add the data to the list, including the current attribute field number
            data.append({'field_number': field_counters[element.tag], 'file_name': file_name, 'cleaned_tag': cleaned_tag, 'tag': element.tag, 'key': key, 'value': value, 'UUID': uuid})

        # Increment the counter for this attribute field
        field_counters[element.tag] += 1

# Convert the list of dictionaries to a DataFrame
df2 = pd.DataFrame(data)

df2

,field_number,file_name,cleaned_tag,tag,key,value,UUID
0,1,xml_file_path,Emisor,{http://www.sat.gob.mx/cfd/3}Emisor,Rfc,EMS2103108P3,None
1,1,xml_file_path,Emisor,{http://www.sat.gob.mx/cfd/3}Emisor,Nombre,ESAB MEXICO SERVICES SA DE CV,None
2,1,xml_file_path,Emisor,{http://www.sat.gob.mx/cfd/3}Emisor,RegimenFiscal,601,None
3,1,xml_file_path,Receptor,{http://www.sat.gob.mx/cfd/3}Receptor,Rfc,AOAR951019842,None
4,1,xml_file_path,Receptor,{http://www.sat.gob.mx/cfd/3}Receptor,Nombre,Roberto Carlos Alonso Alvarez,None
...,...,...,...,...,...,...,...
96,1,xml_file_path,Comprobante,{http://www.sat.gob.mx/cfd/3}Comprobante,Moneda,MXN,B7D29180-3BB4-904C-9DC5-8730016AA924
97,1,xml_file_path,Comprobante,{http://www.sat.gob.mx/cfd/3}Comprobante,Total,28588.97,B7D29180-3BB4-904C-9DC5-8730016AA924
98,1,xml_file_path,Comprobante,{http://www.sat.gob.mx/cfd/3}Comprobante,TipoDeComprobante,N,B7D29180-3BB4-904C-9DC5-8730016AA924
99,1,xml_file_path,Comprobante,{http://www.sat.gob.mx/cfd/3}Comprobante,MetodoPago,PUE,B7D29180-3BB4-904C-9DC5-8730016AA924


In [74]:
# display data using pandasgui
pg.show(df2)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI


In [80]:
for element in elements:
    if element.attrib:
        # Check if the element is 'TimbreFiscalDigital' and then extract the UUID
        if cleaned_tag == 'TimbreFiscalDigital':
            uuid = element.attrib.get('UUID')

print(uuid)


B7D29180-3BB4-904C-9DC5-8730016AA924


In [ ]:

# Define a list to store row data
data = []

# Define a dictionary for the attribute field counters
field_counters = {}

# Variable to store the UUID
uuid = None

# Iterate over elements
for element in elements:
    if element.attrib:
        cleaned_tag = re.sub(r"{.*?}", "", element.tag)
        file_name = os.path.basename('xml_file_path')

        # If this tag has not been seen before, initialize its counter
        if element.tag not in field_counters:
            field_counters[element.tag] = 1

        # Check if the element is 'TimbreFiscalDigital' and then extract the UUID
        if cleaned_tag == 'TimbreFiscalDigital':
            uuid = element.attrib.get('UUID')

        # Iterate over attributes of the element
        for key, value in element.attrib.items():
            # Add the data to the list, including the current attribute field number
            data.append({'field_number': field_counters[element.tag], 'file_name': file_name, 'cleaned_tag': cleaned_tag, 'tag': element.tag, 'key': key, 'value': value, 'UUID': uuid})

        # Increment the counter for this attribute field
        field_counters[element.tag] += 1

# Convert the list of dictionaries to a DataFrame
df2 = pd.DataFrame(data)

df2